<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment

In [3]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.3.0-bin-hadoop3.tgz
# install findspark 
!pip install -q findspark
!pip install -q pyspark

!echo "finished :)"

     |████████████████████████████████| 281.4 MB 50 kB/s 
     |████████████████████████████████| 199 kB 65.0 MB/s 
finished :)


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

!echo "env set :)"

env set :)


## Setup environment variables

In [26]:
!wget -P -q https://github.com/ja8an/big-data/blob/main/in/forestfireinput.csv?raw=true -O forestfireinput.csv
!wget -P -q https://github.com/ja8an/big-data/blob/main/in/bible.txt?raw=true -O bible.txt
!echo "finished"

--2022-10-25 22:06:25--  https://github.com/ja8an/big-data/blob/main/in/forestfireinput.csv?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ja8an/big-data/raw/main/in/forestfireinput.csv [following]
--2022-10-25 22:06:25--  https://github.com/ja8an/big-data/raw/main/in/forestfireinput.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv [following]
--2022-10-25 22:06:25--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTT

## Setup Spark

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark import SparkConf, SparkContext, SparkFiles
conf = SparkConf().setAppName("teste").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)
"spark setup"

'spark setup'

In [ ]:
!git clone https://github.com/ja8an/big-data.git

# Activities

## WordCount

In [ ]:
textFile = sc.textFile("/content/bible.txt")
words = textFile.flatMap(lambda line : line.split(" "))
wordCounts = words.map(lambda word : (word, 1))
count = wordCounts.reduceByKey(lambda a, b: a + b)

count.take(5)

[('God', 2229), ('created', 38), ('', 27159), ('And', 12735), ('earth', 328)]

## Forest Fire Mean

In [ ]:
dataset = sc.textFile("/content/forestfireinput.csv")

lines = dataset.map(lambda line : line.split(","))
lines = lines.map(lambda cols: float(cols[4]))

media = lines.reduce(lambda a, b: a + b)

average = media / lines.count()

average

90.64468085106387

In [ ]:
dataset = sc.textFile("/content/airports.txt")

lines = dataset.map(lambda x : x.split("\t"))
filter = lines.filter(lambda x: x[3] == 'United States' and float(x[6]) > 40.0)
cols = filter.map(lambda x: ','.join(x).upper())
cols.take(1)

['6890,DOWAGIAC MUNICIPAL AIRPORT,DOWAGIAC,UNITED STATES,C91,\\N,41.9929342,-86.1280125,748,-5,U,AMERICA/NEW_YORK']

In [ ]:
dataset = sc.textFile("/content/airports.txt")

lines = dataset.map(lambda x : x.split("\t"))
filter = lines.filter(lambda x: x[3] == 'United States')
cols = filter.map(lambda x: float(x[6]))
sum = cols.reduce(lambda x, y: x + y)
sum / filter.count()

41.44429158866408

In [27]:
from functools import reduce

dataset = sc.textFile("/content/RealState.csv")

lines = dataset.map(lambda x: x.split(',')).filter(lambda x: x[0] != "MLS")

data = lines.map(lambda x: (int(x[3]), float(x[2])))

group = data.groupByKey()

def media(x):
  t = list(x)
  return reduce(lambda x, y: x + y, t) / len(t)

group.mapValues(media).sortByKey().collect()

[(0, 293450.0),
 (1, 169981.81818181818),
 (2, 266356.3739837398),
 (3, 359062.20649651974),
 (4, 483475.6497175141),
 (5, 657858.0645161291),
 (6, 603225.0),
 (7, 325000.0),
 (10, 699000.0)]

In [57]:
import re

dataset = sc.textFile("/content/bible.txt")

comp = re.compile(r"\W+")
rows = dataset.flatMap(lambda x: comp.split(x.lower()))
filt = rows.filter(lambda x: len(x) > 0)
words = filt.map(lambda x: (x, 1))
output = words.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False)

# Da pra misturar comandos do linux com o bloco, é só colocar o ! no começo da linha
# esse comando força a exclusão (f) da pasta recursivamente (-r)
!rm -rf /content/teste
output.saveAsTextFile("/content/teste")